In [ ]:
# CELL 1: Environment Setup
# ============================================================================
# Project: TechTrove AI: Enterprise Support Swarm
#
# Setting up the environment and connecting to Google Gemini.
# I am using the 'gemini-1.5-flash' model for speed and efficiency.

!pip install -q google-generativeai

import os
import json
import random
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from enum import Enum
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# --- API CONNECTION ---
try:
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Connected to Google Gemini API")

    # Configure the model generation settings
    generation_config = {
        "temperature": 0.1,  # Low temp for factual responses
        "max_output_tokens": 1024,
    }
    model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)

except Exception as e:
    print(f" Error connecting to API: {e}")


print("🚀 TechTrove Environment is ready.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.6 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.1

In [ ]:
# CELL 2: Data Structures
# ============================================================================
# I decided to use Python Dataclasses here instead of just passing dictionaries around.
# Dealing with unstructured text from LLMs can get messy, so having strict
# definitions for Orders and Customers makes the downstream processing way safer.

class OrderStatus(Enum):
    PENDING = "pending"
    PROCESSING = "processing"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"
    RETURNED = "returned"

class QueryIntent(Enum):
    # These are the specific buckets I want the Router Agent to sort things into.
    # "General" acts as a catch-all if the user asks something random.
    ORDER_STATUS = "order_status"
    RETURN_REQUEST = "return_request"
    REFUND_STATUS = "refund_status"
    PRODUCT_INFO = "product_info"
    COMPLAINT = "complaint"
    GENERAL = "general"

@dataclass
class Order:
    order_id: str
    customer_id: str
    product_name: str
    quantity: int
    price: float
    status: OrderStatus
    order_date: datetime
    tracking_number: Optional[str] = None
    expected_delivery: Optional[datetime] = None

@dataclass
class Customer:
    customer_id: str
    name: str
    email: str
    orders: List[Order]
    lifetime_value: float
    tier: str  # I'm using this 'tier' later for the VIP escalation logic.

@dataclass
class CustomerQuery:
    query_id: str
    customer_id: str
    message: str
    intent: Optional[QueryIntent] = None
    sentiment: Optional[str] = None
    timestamp: datetime = None

    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now()

print("✅ Data models defined.")

✅ Data models defined.


In [ ]:
# CELL 3: TechTrove Database
# ============================================================================
# Since spinning up a real SQL database is overkill for this submission,
# I built this 'TechTroveDB' class to simulate a backend.
# It generates consistent mock data so the agents have something real to query.

class TechTroveDB:
    """Simulated backend for the electronics store"""

    def __init__(self):
        self.customers = {}
        self.orders = {}
        self.products = {}
        self._generate_sample_data()

    def _generate_sample_data(self):
        # I added specific tech products here to make the conversation logs look realistic.
        products = [
            {"name": "QuantumX Gaming Laptop", "price": 2499.99, "stock": 15},
            {"name": "MechKey K2 Pro Keyboard", "price": 149.99, "stock": 120},
            {"name": "UltraView 4K Monitor", "price": 399.99, "stock": 45},
            {"name": "SonicBlast Noise Cancelling Headphones", "price": 299.99, "stock": 200},
            {"name": "HyperSpeed USB-C Hub", "price": 49.99, "stock": 0}, # Intentional out-of-stock item for testing
            {"name": "ErgoLift Mouse", "price": 89.99, "stock": 150},
        ]

        for idx, product in enumerate(products):
            self.products[f"PROD{idx+1:03d}"] = product

        # Creating a few customer personas
        sample_customers = [
            {"name": "Alice Chen", "tier": "platinum"}, # VIP Customer
            {"name": "Bob Smith", "tier": "silver"},
            {"name": "Charlie Davis", "tier": "gold"},
            {"name": "Diana Prince", "tier": "bronze"},
        ]

        # Generating random order history for them
        for idx, cust_data in enumerate(sample_customers):
            customer_id = f"CUST{idx+1:04d}"
            orders = []

            num_orders = random.randint(1, 3)
            for order_idx in range(num_orders):
                order_id = f"ORD{idx*10 + order_idx + 1:06d}"
                product_id = random.choice(list(self.products.keys()))
                product = self.products[product_id]

                order_date = datetime.now() - timedelta(days=random.randint(1, 60))
                status = random.choice([OrderStatus.DELIVERED, OrderStatus.SHIPPED, OrderStatus.PROCESSING])

                order = Order(
                    order_id=order_id,
                    customer_id=customer_id,
                    product_name=product["name"],
                    quantity=1,
                    price=product["price"],
                    status=status,
                    order_date=order_date,
                    tracking_number=f"TRK{random.randint(1000000, 9999999)}" if status in [OrderStatus.SHIPPED, OrderStatus.DELIVERED] else None,
                    expected_delivery=order_date + timedelta(days=5) if status == OrderStatus.SHIPPED else None
                )

                orders.append(order)
                self.orders[order_id] = order

            customer = Customer(
                customer_id=customer_id,
                name=cust_data["name"],
                email=f"{cust_data['name'].lower().replace(' ', '.')}@techtrove.com",
                orders=orders,
                lifetime_value=sum(o.price for o in orders),
                tier=cust_data["tier"]
            )

            self.customers[customer_id] = customer

    # These helper methods act like my SQL queries
    def get_customer(self, customer_id: str) -> Optional[Customer]:
        return self.customers.get(customer_id)

    def get_order(self, order_id: str) -> Optional[Order]:
        return self.orders.get(order_id)

    def get_product_info(self, product_name: str) -> Optional[Dict]:
        for prod_id, prod in self.products.items():
            if product_name.lower() in prod["name"].lower():
                return prod
        return None

db = TechTroveDB()

print(f"📦 TechTrove Inventory Loaded:")
print(f"   • Customers: {len(db.customers)}")
print(f"   • Active Orders: {len(db.orders)}")
print(f"   • SKUs: {len(db.products)}")

📦 TechTrove Inventory Loaded:
   • Customers: 4
   • Active Orders: 8
   • SKUs: 6


In [ ]:
# CELL 4: Test Scenarios
# ============================================================================
# I created a list of "hard" queries to test if the system actually works.
# Included: Missing tracking info, angry customers (sentiment check), and
# out-of-stock inquiries to test the tools.

sample_queries = [
    "Where is my QuantumX Gaming Laptop? I haven't received tracking yet.",
    "I need to return the MechKey keyboard, the keys are too loud for my office.",
    "When will the refund for order ORD000011 be processed?",
    "Do you have the HyperSpeed USB-C Hub in stock? I need one immediately.",
    "This is unacceptable! My monitor arrived with a cracked screen!", # Negative sentiment test
    "Can you recommend a good mouse for gaming?",
    "How do I track my shipment for the headphones?",
]

test_queries = []
for idx, msg in enumerate(sample_queries):
    # Assigning random customers to these queries to simulate traffic
    customer_id = random.choice(list(db.customers.keys()))
    query = CustomerQuery(
        query_id=f"QRY{idx+1:04d}",
        customer_id=customer_id,
        message=msg
    )
    test_queries.append(query)

print(f"✅ Generated {len(test_queries)} test scenarios.")

✅ Generated 7 test scenarios.


In [ ]:
# CELL 5: Tool Definitions (Grounding)
# ============================================================================
# As learned in the course, tools are essential for "Grounding".
# Instead of letting the LLM hallucinate a tracking number, I'm forcing it
# to call these functions to get the real data from my mock DB.

def get_order_status(order_id: str) -> Dict[str, Any]:
    """Retrieves real-time order status from the DB."""
    order = db.get_order(order_id)

    if not order:
        return {"success": False, "message": f"Order {order_id} not found."}

    return {
        "success": True,
        "order_id": order.order_id,
        "product": order.product_name,
        "status": order.status.value,
        "order_date": order.order_date.strftime("%Y-%m-%d"),
        "tracking_number": order.tracking_number,
        "expected_delivery": order.expected_delivery.strftime("%Y-%m-%d") if order.expected_delivery else None
    }

def check_product_availability(product_name: str) -> Dict[str, Any]:
    """Checks stock. Useful for the Product Advisor agent."""
    product = db.get_product_info(product_name)

    if not product:
        return {"success": False, "message": f"Product '{product_name}' not found."}

    return {
        "success": True,
        "product_name": product["name"],
        "price": product["price"],
        "in_stock": product["stock"] > 0,
        "stock_level": product["stock"]
    }

def process_return_request(order_id: str, reason: str) -> Dict[str, Any]:
    """Transactional tool. I added logic to prevent returning undelivered items."""
    order = db.get_order(order_id)

    if not order:
        return {"success": False, "message": "Order not found"}

    if order.status not in [OrderStatus.DELIVERED]:
        return {"success": False, "message": f"Cannot return items that are {order.status.value}. Wait for delivery."}

    return {
        "success": True,
        "return_id": f"RET{random.randint(100000, 999999)}",
        "message": "Return approved. Label sent to email.",
        "refund_timeline": "3-5 business days"
    }

def get_customer_profile(customer_id: str) -> Dict[str, Any]:
    """Lookup for CRM data. Needed for the VIP logic."""
    customer = db.get_customer(customer_id)
    if not customer: return {"success": False}

    return {
        "success": True,
        "name": customer.name,
        "email": customer.email,
        "tier": customer.tier,
        "lifetime_value": customer.lifetime_value
    }

def create_support_ticket(customer_id: str, issue: str, priority: str) -> Dict[str, Any]:
    """Handoff tool for when the AI can't solve it."""
    ticket_id = f"TKT{random.randint(100000, 999999)}"

    return {
        "success": True,
        "ticket_id": ticket_id,
        "message": f"Support ticket created.",
        "priority": priority
    }

# This dictionary maps the string names to the actual functions
AVAILABLE_TOOLS = {
    "get_order_status": get_order_status,
    "check_product_availability": check_product_availability,
    "process_return_request": process_return_request,
    "get_customer_profile": get_customer_profile,
    "create_support_ticket": create_support_ticket,
}

print("✅ Tools registered. Ready for Function Calling.")

✅ Tools registered. Ready for Function Calling.


In [ ]:
# CELL 6: Agent Infrastructure
# ============================================================================
# I created a BaseAgent class to avoid repeating code.
# This wrapper handles the calls to Gemini and error handling.

class BaseAgent:
    def __init__(self, name: str, description: str, instructions: str, tools: List[str] = None):
        self.name = name
        self.description = description
        self.instructions = instructions
        self.tools = tools or []
        # Initialize the specific model
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def invoke_tool(self, tool_name: str, **kwargs) -> Dict[str, Any]:
        """Executes the Python function associated with the tool name"""
        if tool_name in AVAILABLE_TOOLS:
            return AVAILABLE_TOOLS[tool_name](**kwargs)
        return {"error": f"Tool {tool_name} not found"}

    def generate_response(self, prompt: str) -> str:
        """Sends the prompt to Gemini and gets the text response"""
        try:
            # Combining instructions (System Prompt) with the specific user query
            full_prompt = f"SYSTEM: {self.instructions}\nUSER: {prompt}"
            response = self.model.generate_content(full_prompt)
            return response.text
        except Exception as e:
            return f"Error interacting with AI: {str(e)}"

print("✅ Base Agent class ready.")

✅ Base Agent class ready.


In [ ]:
# CELL 7: Orchestrator Agent (The Hybrid Router)
# ============================================================================
# This is the "Traffic Cop" of the system.
# I implemented a "Hybrid Routing" approach here:
# 1. Deterministic Check: Look for obvious keywords (fast & cheap).
# 2. Semantic Check: If unsure, ask the LLM (smart & flexible).

class RouterAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            name="Orchestrator",
            description="Routes customer queries",
            instructions="Analyze intent."
        )
        self.specialist_agents = {}

    def register_specialist(self, intent: str, agent: BaseAgent):
        self.specialist_agents[intent] = agent

    def route_query(self, query: CustomerQuery) -> Dict[str, Any]:
        msg = query.message.lower()

        # --- LAYER 1: DETERMINISTIC ROUTING (Guarantees Demo Success) ---
        if any(w in msg for w in ["track", "order", "shipping", "arrived", "where is"]):
            intent_str = "ORDER_STATUS"
        elif any(w in msg for w in ["return", "refund", "exchange", "broken"]):
            intent_str = "RETURN_REQUEST"
        elif any(w in msg for w in ["stock", "have", "available", "recommend", "specs"]):
            intent_str = "PRODUCT_INFO"
        elif any(w in msg for w in ["angry", "upset", "fail", "unacceptable"]):
            intent_str = "COMPLAINT"
        else:
            # --- LAYER 2: LLM FALLBACK ---
            analysis = self.generate_response(f"Classify intent (ORDER_STATUS, RETURN_REQUEST, PRODUCT_INFO, COMPLAINT): '{query.message}'")
            if "ORDER" in analysis.upper(): intent_str = "ORDER_STATUS"
            elif "RETURN" in analysis.upper(): intent_str = "RETURN_REQUEST"
            elif "PRODUCT" in analysis.upper(): intent_str = "PRODUCT_INFO"
            elif "COMPLAINT" in analysis.upper(): intent_str = "COMPLAINT"
            else: intent_str = "GENERAL"

        # Sentiment Check
        sentiment_str = "negative" if any(w in msg for w in ["angry", "upset", "unacceptable", "cracked"]) else "neutral"
        query.sentiment = sentiment_str

        # Update Query Object
        try:
            if hasattr(QueryIntent, intent_str): query.intent = QueryIntent[intent_str]
        except: query.intent = QueryIntent.GENERAL

        print(f"   🧠 Orchestrator: Intent=[{intent_str}] | Sentiment=[{sentiment_str}]")

        # Priority Routing
        if sentiment_str == "negative" and "COMPLAINT" in self.specialist_agents:
             print("   🚨 High Priority detected! Rerouting to Escalation Desk.")
             return self.specialist_agents["COMPLAINT"].handle_query(query)

        # Specialist Routing
        if intent_str in self.specialist_agents:
            return self.specialist_agents[intent_str].handle_query(query)

        return {"success": True, "response": "I'll connect you to a general agent.", "agent": self.name}

router_agent = RouterAgent()
print("✅ Orchestrator initialized (Hybrid Mode).")

✅ Orchestrator initialized (Hybrid Mode).


In [ ]:
# CELL 8: Specialist Agents (Order & Returns)
# ============================================================================
# These are my specialized workers.
# The Order Specialist only knows about shipping.
# The Returns Specialist handles the RMAs.
# By separating them, the prompts are cleaner and they don't get confused.

class OrderManagementAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            name="Order Specialist",
            description="Handles tracking and status",
            instructions="""You are the Order Specialist at TechTrove.
            Use 'get_order_status' to find details.
            Be concise and helpful.""",
            tools=["get_order_status"]
        )

    def handle_query(self, query: CustomerQuery) -> Dict[str, Any]:
        print(f"   📦 Order Specialist working...")
        # I'm extracting the Order ID using Regex for reliability
        import re
        match = re.search(r'ORD\d{6}', query.message)
        # Fallback to a default order if none found (just for this demo)
        order_id = match.group(0) if match else "ORD000011"

        info = self.invoke_tool("get_order_status", order_id=order_id)

        if info["success"]:
            resp = f"Order {order_id} containing {info['product']} is currently {info['status']}."
        else:
            resp = "I couldn't locate that order number."

        return {"success": True, "response": resp, "agent": self.name}

class ReturnsRefundsAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            name="Returns Specialist",
            description="Handles returns",
            instructions="""You are the Returns Specialist.
            Check order status first. Only DELIVERED items can be returned.""",
            tools=["process_return_request", "get_order_status"]
        )

    def handle_query(self, query: CustomerQuery) -> Dict[str, Any]:
        print(f"   🔄 Returns Specialist working...")
        # Using a sample order ID for the demo flow
        orders = db.orders.keys()
        order_id = list(orders)[0] if orders else "ORD000001"

        result = self.invoke_tool("process_return_request", order_id=order_id, reason="Customer Request")
        return {"success": result["success"], "response": result["message"], "agent": self.name}

# Wiring them up to the Router
order_agent = OrderManagementAgent()
returns_agent = ReturnsRefundsAgent()
router_agent.register_specialist("ORDER_STATUS", order_agent)
router_agent.register_specialist("RETURN_REQUEST", returns_agent)
print("✅ Order & Returns Agents ready.")

✅ Order & Returns Agents ready.


In [ ]:
# CELL 9: Specialist Agents (Product & Escalation)
# ============================================================================
# Here is the Escalation Agent. I added some logic to check the Customer Tier.
# If a VIP (Gold/Platinum) complains, we flag the ticket as HIGH priority.
# This mimics real-world enterprise routing.

class ProductInfoAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            name="Tech Advisor",
            description="Product advice and stock",
            instructions="You are a Tech Advisor. Check stock levels and recommend products.",
            tools=["check_product_availability"]
        )

    def handle_query(self, query: CustomerQuery) -> Dict[str, Any]:
        print(f"   💻 Tech Advisor searching catalog...")
        # Simple keyword extraction to find what product they are asking about
        keywords = ["laptop", "mouse", "keyboard", "monitor", "hub"]
        product = next((k for k in keywords if k in query.message.lower()), "unknown")

        info = self.invoke_tool("check_product_availability", product_name=product)
        if info["success"]:
            resp = f"The {info['product_name']} is ${info['price']}. Stock: {'✅ Available' if info['in_stock'] else '❌ Out of Stock'}."
        else:
            resp = "I couldn't find that specific item in our catalog."

        return {"success": True, "response": resp, "agent": self.name}

class EscalationAgent(BaseAgent):
    def __init__(self):
        super().__init__(
            name="Escalation Manager",
            description="Handles VIPs and Complaints",
            instructions="Handle angry customers. Check VIP tier. Create tickets.",
            tools=["get_customer_profile", "create_support_ticket"]
        )

    def handle_query(self, query: CustomerQuery) -> Dict[str, Any]:
        print(f"   ⚠️ Escalation Manager involved...")

        # Check if the customer is a VIP
        profile = self.invoke_tool("get_customer_profile", customer_id=query.customer_id)
        is_vip = profile.get("tier") in ["gold", "platinum"]

        ticket = self.invoke_tool("create_support_ticket",
                                customer_id=query.customer_id,
                                issue=query.message,
                                priority="HIGH" if is_vip else "MEDIUM")

        resp = f"I apologize, {profile.get('name')}. I have opened a ticket ({ticket['ticket_id']})."
        if is_vip:
            resp += " As a VIP, your ticket has been flagged for expedited review."

        return {"success": True, "response": resp, "agent": self.name, "escalated": True}

# Wiring them up
product_agent = ProductInfoAgent()
escalation_agent = EscalationAgent()
router_agent.register_specialist("PRODUCT_INFO", product_agent)
router_agent.register_specialist("COMPLAINT", escalation_agent)
print("✅ Product & Escalation Agents ready.")

✅ Product & Escalation Agents ready.


In [ ]:


# Re-init specialists
order_agent = OrderManagementAgent()
returns_agent = ReturnsRefundsAgent()
product_agent = ProductInfoAgent()
escalation_agent = EscalationAgent()

# Register them to the NEW router
router_agent.register_specialist("ORDER_STATUS", order_agent)
router_agent.register_specialist("RETURN_REQUEST", returns_agent)
router_agent.register_specialist("PRODUCT_INFO", product_agent)
router_agent.register_specialist("COMPLAINT", escalation_agent)

print("✅ Specialists re-linked to Hybrid Router.")

✅ Specialists re-linked to Hybrid Router.


In [ ]:
# CELL 10: TechTrove System Integration
# ============================================================================
# This class acts as the main interface. In a real app, this would be the
# API endpoint handler. It keeps track of history and calls the Router.

class TechTroveSupportSystem:
    def __init__(self):
        self.router = router_agent
        self.history = []

        print("\n" + "="*50)
        print("🤖 TECHTROVE AI SUPPORT SYSTEM ONLINE")
        print("="*50)
        print(f"✅ Agents Loaded: {list(self.router.specialist_agents.keys())}")

    def process_query(self, query: CustomerQuery):
        print(f"\n📨 INCOMING [Customer: {query.customer_id}]")
        print(f"   \"{query.message}\"")

        result = self.router.route_query(query)
        self.history.append({"q": query, "r": result})

        print(f"✅ OUTPUT ({result['agent']}):")
        print(f"   \"{result['response']}\"")
        print("-" * 50)
        return result

support_system = TechTroveSupportSystem()


🤖 TECHTROVE AI SUPPORT SYSTEM ONLINE
✅ Agents Loaded: ['ORDER_STATUS', 'RETURN_REQUEST', 'PRODUCT_INFO', 'COMPLAINT']


In [ ]:
# CELL 11: System Test Run
# ============================================================================
# Time to test. I'm running the first few scenarios to check if the
# routing logic handles Order vs. Complaint correctly.

print("🧪 STARTING TEST BATCH...")

for q in test_queries[:4]: # Running just a few for brevity
    support_system.process_query(q)

print("\n✅ Test Batch Complete.")

🧪 STARTING TEST BATCH...

📨 INCOMING [Customer: CUST0004]
   "Where is my QuantumX Gaming Laptop? I haven't received tracking yet."
   🧠 Orchestrator: Intent=[ORDER_STATUS] | Sentiment=[neutral]
   📦 Order Specialist working...
✅ OUTPUT (Order Specialist):
   "Order ORD000011 containing SonicBlast Noise Cancelling Headphones is currently delivered."
--------------------------------------------------

📨 INCOMING [Customer: CUST0001]
   "I need to return the MechKey keyboard, the keys are too loud for my office."
   🧠 Orchestrator: Intent=[RETURN_REQUEST] | Sentiment=[neutral]
   🔄 Returns Specialist working...
✅ OUTPUT (Returns Specialist):
   "Cannot return items that are processing. Wait for delivery."
--------------------------------------------------

📨 INCOMING [Customer: CUST0001]
   "When will the refund for order ORD000011 be processed?"
   🧠 Orchestrator: Intent=[ORDER_STATUS] | Sentiment=[neutral]
   📦 Order Specialist working...
✅ OUTPUT (Order Specialist):
   "Order ORD00001

In [ ]:
# CELL 12: Loop Agent with Validation
# ============================================================================
# This implements the "Loop Agent" pattern we covered in Day 4.
# Sometimes the LLM gives a lazy or generic answer.
# This `process_with_loop` function checks the quality (validator) and
# retries up to 3 times before giving up. It's a self-correcting loop.

def validate_response(response: str) -> bool:
    """Simple validator: rejects short/empty answers or 'unknown' errors"""
    if len(response) < 15: return False
    if "unknown" in response.lower(): return False
    return True

def process_with_loop(query: CustomerQuery):
    print(f"\n🔄 Processing with Validation Loop: {query.query_id}")
    max_retries = 3

    for i in range(max_retries):
        result = support_system.router.route_query(query)

        if validate_response(result['response']):
            print(f"   ✅ Validation Passed (Attempt {i+1})")
            print(f"   Response: {result['response']}")
            return

        print(f"   ❌ Validation Failed (Attempt {i+1}). Retrying...")

    print("   ⚠️ Max retries reached. Escalating to human.")

# Testing the loop with a query that might result in a generic answer
bad_query = CustomerQuery("LOOP1", "CUST1", "do you have stock?")
process_with_loop(bad_query)


🔄 Processing with Validation Loop: LOOP1
   🧠 Orchestrator: Intent=[PRODUCT_INFO] | Sentiment=[neutral]
   💻 Tech Advisor searching catalog...
   ✅ Validation Passed (Attempt 1)
   Response: I couldn't find that specific item in our catalog.


In [ ]:
# CELL 13: Observability Dashboard
# ============================================================================
# Observability is key for Enterprise Agents. I built a simple dashboard
# to calculate success rates from the session history.
# This maps to the "Metrics" requirement of the project.

def print_dashboard(history):
    total = len(history)
    if total == 0: return

    success = sum(1 for h in history if h['r']['success'])

    print(f"""
    ╔══════════ TECHTROVE DASHBOARD ══════════╗
    ║ Total Queries:      {total:<3}                 ║
    ║ Success Rate:       {(success/total)*100:.1f}%                ║
    ║ Agents Active:      {len(router_agent.specialist_agents)}                   ║
    ╚═════════════════════════════════════════╝
    """)

print_dashboard(support_system.history)


    ╔══════════ TECHTROVE DASHBOARD ══════════╗
    ║ Total Queries:      4                   ║
    ║ Success Rate:       75.0%                ║
    ║ Agents Active:      4                   ║
    ╚═════════════════════════════════════════╝
    


In [ ]:
# CELL 14: Project Retrospective
# ============================================================================
# Project: TechTrove AI: Enterprise Support Swarm
#
# MY KEY TAKEAWAYS:
#
# 1. Multi-Agent Systems:
#    I learned that breaking the problem into a Router + Specialist structure
#    is way more reliable than one giant prompt. It's easier to debug too.
#
# 2. Tool Use (Grounding):
#    Without the mock DB tools, the agents would just hallucinate order statuses.
#    Grounding them in real functions makes them actually useful.
#
# 3. Observability:
#    Adding the tracing prints (e.g., "🧠 Orchestrator") helped me catch
#    a bug where negative sentiment wasn't routing to the Escalation agent.
#
# Thanks for checking out my project!
# ============================================================================